In [1]:
# Import the required libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import joblib

In [2]:
tama_df = pd.read_csv('engineered_tama_data.csv')
tama_df

,hebrew_city_name,city_code,year,total_open_requests,total_requests,total_approved_requests
0,אור יהודה,2400,2005,0,0.0,0.0
1,אור יהודה,2400,2006,0,0.0,0.0
2,אור יהודה,2400,2007,0,0.0,0.0
3,אור יהודה,2400,2008,0,1.0,0.0
4,אור יהודה,2400,2009,1,0.0,0.0
...,...,...,...,...,...,...
1075,תל אביב יפו,5000,2018,687,164.0,83.0
1076,תל אביב יפו,5000,2019,768,116.0,149.0
1077,תל אביב יפו,5000,2020,735,238.0,115.0
1078,תל אביב יפו,5000,2021,858,240.0,124.0


### prepare data

In [3]:
tama_df['city_code'] = tama_df['city_code'].astype("category")
# Normalize the 'year' column
tama_df['year'] = tama_df['year'] - 2005

In [4]:
tama_df['year'].min()

0

In [5]:
tama_df['year'].max()

17

In [34]:
# split it into training and testing sets
features_list = ['city_code', 'year', 'total_open_requests']
test_cities = ['כפר סבא', 'נתניה']
train_df = tama_df[(tama_df['year'] < 14) & (~tama_df['hebrew_city_name'].isin(test_cities))]
test_df = tama_df[(tama_df['year'] >= 14) & (tama_df['hebrew_city_name'].isin(test_cities))]

X_train, y_train = train_df[features_list], train_df['total_approved_requests']
X_test, y_test = test_df[features_list], test_df['total_approved_requests']

In [35]:
import numpy as np
# Instantiate the Random Forest Regressor with desired parameters
rf_regressor = RandomForestRegressor(n_estimators=10, random_state=42)

# Fit the model to the training data
rf_regressor.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_regressor.predict(X_test)

# Evaluate the model using mean squared error
mse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mean Squared Error:", mse)

Mean Squared Error: 12.256069924735254


### Save the model to a file

In [36]:
import joblib
joblib.dump(rf_regressor, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [37]:
rf_regressor = joblib.load('random_forest_model.pkl')

In [32]:
tama_df['pred_total_approved_requests'] = rf_regressor.predict(
    tama_df[['city_code', 'year', 'total_open_requests']])
tama_df = tama_df[
    ['city_code', 'hebrew_city_name', 'year', 'total_requests', 'total_open_requests', 'total_approved_requests', 'pred_total_approved_requests']]
tama_df

,city_code,hebrew_city_name,year,total_requests,total_open_requests,total_approved_requests,pred_total_approved_requests
0,2400,אור יהודה,0,0.0,0,0.0,0.0
1,2400,אור יהודה,1,0.0,0,0.0,0.0
2,2400,אור יהודה,2,0.0,0,0.0,0.0
3,2400,אור יהודה,3,1.0,0,0.0,0.0
4,2400,אור יהודה,4,0.0,1,0.0,0.3
...,...,...,...,...,...,...,...
1075,5000,תל אביב יפו,13,164.0,687,83.0,83.0
1076,5000,תל אביב יפו,14,116.0,768,149.0,83.0
1077,5000,תל אביב יפו,15,238.0,735,115.0,83.0
1078,5000,תל אביב יפו,16,240.0,858,124.0,83.0


### plot

In [38]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_style("whitegrid")

from bidi.algorithm import get_display

tama_df_display_df = tama_df.drop_duplicates()
tama_df_display_df['hebrew_city_name'] = tama_df_display_df['hebrew_city_name'].apply(get_display)
tama_df_display_df['year'] = tama_df_display_df['year'] + 5
tama_df_display_df['year'] = tama_df_display_df['year'].astype(int)
sns.set(font_scale=3) 



tama_df_display_df['pred_total_approved_requests'] = tama_df_display_df['pred_total_approved_requests']

g = sns.FacetGrid(tama_df_display_df, col="hebrew_city_name", col_wrap=3, height=10, ylim=(0, 150))
g.map(plt.plot, "year", "pred_total_approved_requests", linestyle="-", marker="o", color="blue")
g.map(plt.plot, "year", "total_approved_requests", linestyle="-", marker="o", color="green")
g.set_titles('{col_name}')
g.set_xlabels("years")
g.set_ylabels("total approved requests")
g.set(xticks=[5, 10, 15, 20], yticks=list(range(0, 151))[::30])
# g.savefig('plot_random_forest_model.png')

In [12]:
# import seaborn as sns
# from matplotlib import pyplot as plt

# sns.set_style("ticks")

# from bidi.algorithm import get_display

# for city in tama_df['hebrew_city_name'].unique():
#     city_df = tama_df[tama_df['hebrew_city_name'] == city]
#     city_df['year'] = city_df['year'].astype(int) + 2005
#     plt.figure(figsize=(10, 6))
#     sns.set(font_scale=1.5)
#     sns.lineplot(data=city_df, x='year', y='total_approved_requests', color='green', label='Real Data')
#     sns.lineplot(data=city_df, x='year', y='pred_total_approved_requests', color='blue', label='Random Forest Model Prediction')
#     plt.title(get_display(city))
#     plt.xlabel('Years')
#     plt.ylabel('Total Approved Requests')
#     plt.xticks([2005, 2010, 2015, 2020])
#     plt.yticks(range(0, 151, 30))
#     plt.savefig(f'cities_plot_random_forest_model/{city}.png')
#     plt.close()